<a href="https://colab.research.google.com/github/ctclumak/posts/blob/master/Google_Scripting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
pip install fake-useragent

  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-cp36-none-any.whl size=13484 sha256=677e70af0a0bff4736b390e9f611be81ef866e2dd63e5cc046c7bbfe5e3110ee
  Stored in directory: /root/.cache/pip/wheels/5e/63/09/d1dc15179f175357d3f5c00cbffbac37f9e8690d80545143ff
Successfully built fake-useragent


**Import libraries**

In [0]:
import urllib
import requests
import pandas as pd
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import re
import urllib.parse
from urllib.parse import urlparse
import randomAgent

**Define the Query, Result number and Exclude URL**

In [0]:
query = "Jeep"  ## The nested quotation mark is to ensure "phrase match" 
number_result = 20
exclude_url = ['youtube', "jeep", "fca"] # You may add more exclusions

query = query.replace(' ', '+')
for exclude in exclude_url:
    query = query + " -inurl:" + exclude

query = urllib.parse.quote_plus(query) # Format into URL encoding


In [0]:

##Fetch the url==================
ua = UserAgent()

google_url = "https://www.google.com/search?q={query}" + "&num=" + str(number_result)
response = requests.get(google_url, {"User-Agent": ua.random}) #ua.random is used to vary all the user agents scraping the websites

if (response.status_code ==200):
  soup = BeautifulSoup(response.text, "html.parser") ##After fetching the response if it returns a status code of 200 start scraping using Beautiful Soup.


##=================
result_div = soup.find_all('div', attrs = {'class': 'ZINbbc'})  ## Check the google SERP to find out the class for your result, you might need to change it here

links = []
titles = []
descriptions = []   
for r in result_div:
    try:
        link = r.find('a', href = True)
        title = r.find('div', attrs={'class':'vvjwJb'}).get_text()  ## Check the google SERP to find out the class for your result, you might need to change it here
        description = r.find('div', attrs={'class':'s3v9rd'}).get_text()  ## you might need to change the div and class here, inspect the page you script 
        # Check to make sure everything is present before appending
        
        if link != '' and title != '' and description != '':
            links.append(link['href'])
            titles.append(title)
            descriptions.append(description)
    except:
        continue

In [0]:
## Remove google news search result
import re   

to_remove = []
clean_links = []
for i, l in enumerate(links):
    clean = re.search('\/url\?q\=(.*)\&sa',l)

    # Anything that doesn't fit the above pattern will be removed
    if clean is None:
        to_remove.append(i)
        continue
    clean_links.append(clean.group(1))

# Remove the corresponding titles & descriptions
for x in to_remove:
    del titles[x]
    del descriptions[x]

In [0]:
import pandas as pd

output_dict = {
'Title': titles,
'Description': descriptions,
'URL': clean_links,
}
output_df = pd.DataFrame(output_dict, columns = output_dict.keys())

#output_df.to_csv('file_name.csv', encoding='utf-8')

In [0]:
output_df

,Title,Description,URL
0,"Jeep Incentives, Deals, & Lease Offers. Find Y...","Find the best local incentives, offers and lea...",https://www.jeepcurrentoffers.com/
1,Southern Chrysler Jeep - Greenbrier | Chrysler...,The dealership is owned by the same locals who...,https://www.southernchryslerjeepgreenbrier.com/
2,New Chrysler & Jeep Inventory for Sale | Park ...,Are you looking for a great deal? Browse our l...,https://www.parkjeepchrysler.com/new-vehicles/
3,"Chapman Chrysler Jeep Henderson, NV | Las Vega...",Chapman Chrysler Jeep offers new and used Chry...,https://www.chapmanchryslerjeep.com/
4,"Chrysler, Dodge, Jeep and Ram Dealer Paramus N...","Chrysler Jeep Dodge of Paramus in Paramus, NJ ...",https://www.chryslerdodgejeepofparamus.com/
5,White Plains Chrysler Jeep Dodge Dealer | New ...,We are a new and used car dealer in Westcheste...,https://www.whiteplainschrysler.com/
6,San Antonio Chrysler Dodge Jeep Ram: New & Use...,"No matter what your driving need is, we can he...",https://www.sanantoniododgechryslerjeepram.com/
7,Hemet CDJR: New & Used CDJR Dealer | Located i...,"Purchase or Lease a New Chrysler, Dodge, Jeep,...",https://www.hemetcdjr.com/
8,Leith Chrysler Jeep: Chrysler Dealer in Raleig...,Leith Chrysler Jeep is your source for new Chr...,https://www.leithchryslerjeep.com/
9,"Chrysler, Jeep Dealership Braintree MA | Used ...",Quirk Chrysler Jeep is a dealership located ne...,https://www.quirkchryslerjeep.com/
